In [327]:
import os
import sys
import time
import json
import copy
import requests
import datetime

import numpy as np
import pandas as pd
import geopandas as gpd

import networkx as nx

from shapely.geometry import LineString, Point

In [328]:
# convert CRS
import pyproj

CRS_SVY21 = 'PROJCS["SVY21",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",28001.642],PARAMETER["False_Northing",38744.572],PARAMETER["Central_Meridian",103.833333333333],PARAMETER["Scale_Factor",1.0],PARAMETER["Latitude_Of_Origin",1.36666666666667],UNIT["Meter",1.0]]'
CRS_SVY21 = pyproj.CRS.from_string(CRS_SVY21)

# WGS84: EPSG:4326

In [329]:
path = r'station_line_list\station_list.xlsx'
sheet_name = r'2023-01-23'
mrt_route = pd.read_excel(path, sheet_name)
mrt_route

,Line Name,Type,Sequence,Station Code,Station Name,Transfer,Build Start,Opening Date,Closed Date,Lantitude,Longitude,Others
0,Bukit Panjang LRT Line,LRT,1,BP1/NS4,Choa Chu Kang,Standard,NaN,Before 2020,NaN,NaN,NaN,NaN
1,Bukit Panjang LRT Line,LRT,2,BP2,South View,NaN,NaN,Before 2020,NaN,NaN,NaN,NaN
2,Bukit Panjang LRT Line,LRT,3,BP3,Keat Hong,NaN,NaN,Before 2020,NaN,NaN,NaN,NaN
3,Bukit Panjang LRT Line,LRT,4,BP4,Teck Whye,NaN,NaN,Before 2020,NaN,NaN,NaN,NaN
4,Bukit Panjang LRT Line,LRT,5,BP5,Phoenix,NaN,NaN,Before 2020,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
215,Thomson-East Coast Line,MRT,18,TE18,Maxwell,NaN,NaN,2022-11-13 00:00:00,NaN,NaN,NaN,NaN
216,Thomson-East Coast Line,MRT,19,TE19,Shenton Way,NaN,NaN,2022-11-13 00:00:00,NaN,NaN,NaN,NaN
217,Thomson-East Coast Line,MRT,20,CE2/NS27/TE20,Marina Bay,Standard,NaN,2022-11-13 00:00:00,NaN,NaN,NaN,NaN
218,Thomson-East Coast Line,MRT,21,TE21,Marina South,NaN,NaN,None,NaN,NaN,NaN,NaN


# 1 Station Location

## 1.1 Location From DataMall static data

**To be completed**

Data from DataMall static data, [website](https://datamall.lta.gov.sg/content/dam/datamall/datasets/Geospatial/TrainStation.zip)

In [ ]:
path = r"zip://TrainStation_Jan2022.zip!TrainStation_Jan2022/MRTLRTStnPtt.shp"
stn_gpd = gpd.read_file(path)

In [ ]:
# handle the ".shp"
stn_gpd['all_station_codes'] = ''

for row_ix, row in stn_gpd.iterrows():
    
    stn_code_li = stn_gpd.loc[row_ix, 'STN_NO'].split('/')
    
    if set(stn_code_li).issubset(set(CLOSED_STN_CODES)):
        # print(stn_code_li)
        stn_gpd.loc[row_ix, 'all_station_codes'] = 'closed_station'
    else:
        # find the 
        stn_code = stn_code_li[0]
        ix = [stn_code in x for x in TRANSFER_STN_CODE]
        ix = ix.index(True)  
        # print(ix)
        stn_gpd.loc[row_ix, 'all_station_codes'] = ' '.join(TRANSFER_STN_CODE[ix])

# drop closed station from "stn_gpd"
stn_gpd = stn_gpd[~ (stn_gpd['all_station_codes'] == 'closed_station')]

# the location transfer station will be the center of 
# the same name stations in the between lines
stn_gpd = stn_gpd.dissolve('all_station_codes', aggfunc='first', as_index=False)
stn_gpd['geometry'] = stn_gpd['geometry'].centroid
stn_gpd = stn_gpd.reset_index(drop=True)

stn_gpd.plot()

## 1.2 Location From citylines

Data from **citylines**: [website](https://www.citylines.co/singapore)

In [331]:
path = r"zip://geographic_topology/citylines_2023-01-24.zip!singapore_stations.geojson"
station = gpd.read_file(path)

def split_info(series):
    series = series.values[0]
    series = json.loads(series)[0]
    series = pd.Series(series)
    series = series[['line', 'system']]
    return series

station[['line', 'system']] = station[['lines']].apply(split_info, result_type='expand', axis=1)
station = station.drop(['lines'], axis=1)

# print(station['line'].drop_duplicates())
# station = station[station['line'] != 'Sentosa Express']
# station = station[station['line'] != 'Shuttle Tebrau']
# station = station[station['line'] != 'North-South Line']
# station = station[station['line'] != ' Changi Airport Skytrain']

station = station[['name', 'geometry']].dissolve(by='name', as_index=False)

In [333]:
mrt_station = mrt_route[['Station Name', 'Opening Date']].copy()
# Drop haven't opened station (i.e., 'Opening Date' == 'None')
mrt_station = mrt_station.replace(to_replace='None', value=pd.NA)
mrt_station = mrt_station.dropna(subset=['Opening Date'])
mrt_station = mrt_station.drop(['Opening Date'], axis=1)

mrt_station = mrt_station.drop_duplicates(ignore_index=True)

mrt_station = mrt_station.merge(station, how='left', left_on='Station Name', right_on='name', validate='one_to_one')
mrt_station = gpd.GeoDataFrame(mrt_station[['Station Name']], 
                               geometry=mrt_station['geometry'],
                               crs='EPSG:4326')

print(mrt_station[mrt_station['Station Name'].isna()])

mrt_station['geometry'] = mrt_station.to_crs(CRS_SVY21).centroid.to_crs('EPSG:4326')
mrt_station['Lantitude'] = mrt_station['geometry'].y
mrt_station['Longitude'] = mrt_station['geometry'].x
mrt_station = mrt_station.drop(['geometry'], axis=1)

mrt_station = mrt_route.merge(mrt_station, on='Station Name', how='left', validate='many_to_one')
mrt_station = mrt_station.sort_values(['Type', 'Line Name', 'Sequence'])

# mrt_station.to_csv('mrt_station_with_location.csv', index=False)

Empty GeoDataFrame
Columns: [Station Name, geometry]
Index: []


## 1.3 Location From Onemap API

In [71]:
class request_metro_staion():
    
    def __init__(self):
        
        self.url = 'https://developers.onemap.sg/commonapi/search'
        self.params = {'searchVal'      : '', 
                       'returnGeom'     : 'Y',
                       'getAddrDetails' : 'Y',
                       'pageNum'        : ''}
    # ---------------------------------------------------------
    def request_one_page(self, search_value, page_num):
        
        self.params['searchVal'] = search_value
        self.params['pageNum'] = page_num
        
        time.sleep(2.)
        req = requests.get(self.url, params=self.params)
        
        content = json.loads(req.content)
        
        return content['results'], content['totalNumPages'], content['found']
    # ---------------------------------------------------------
    def request_all(self):
        '''
        request all by using key words {'mrt station', 'lrt station'}
        '''
        
        result_all = []
        
        for station_type in ['mrt station', 'lrt station']:
            
            result_type = []
            
            _, total_page, total_result = self.request_one_page(search_value=station_type, page_num=1)
            
            for page_num in range(1, 1+total_page):
                
                res, _, _ = self.request_one_page(search_value=station_type, page_num=page_num)
        
                result_type.extend(res) 
            
            assert len(result_type) == int(total_result)
            
            result_all.extend(copy.deepcopy(result_type))
        
        result_all = pd.DataFrame(result_all)
        
        return result_all
    # --------------------------------------------------------
    def request_by_station_name(self, data, name_col):
        '''
        data (pandas.DataFrame):
            'station_name' column indicates the search name
        name_col (str):
            
        '''
        all_res = []
        
        for search_val in data[name_col].to_list():
            
            res, _, _ = self.request_one_page(search_value=search_val, page_num=1)
            print(res)
            
            res = res[0]
            res[name_col] = search_val
            
            all_res.append(res)
            
        return pd.DataFrame(all_res)
# ==========================================================

In [ ]:
'''
# Request All MRT Station and LRT Station

result_all = request_metro_staion().request_all()

dt = datetime.datetime.now()
path = r'data/metro_staion_{0:0>4}-{1:0>2}-{2:0>2}.csv'.format(dt.year, dt.month, dt.day)
result_all.to_csv(path, index_col=False)
'''


'''
# Request MRT Station and LRT Station according to Station Name

req_station = mrt_route[['Type', 'Station Code', 'Station Name', 'Opening Date']].copy()

req_station.replace(to_replace='None', value=pd.NA, inplace=True)
req_station = req_station.dropna(subset=['Opening Date'])

req_station = req_station.drop_duplicates().reset_index(drop=True)
req_station['search_name'] = req_station['Station Name'].str.cat(req_station['Type'], sep=' ') + ' Station'

res = request_metro_staion()
all_res = res.request_by_station_name(req_station, 'search_name')
# print(all_res)

req_station = req_station.merge(all_res, how='outer', on='search_name')

dt = datetime.datetime.now()
path = r'data/metro_staion_{0:0>4}-{1:0>2}-{2:0>2}.csv'.format(dt.year, dt.month, dt.day)
req_station.to_csv('metro_station.csv')
'''

# 2 Metro Line

## 2.2 Metro Topology From citylines

In [336]:
path = r"zip://geographic_topology/citylines_2023-01-24.zip!singapore_sections.geojson"
mrt_line = gpd.read_file(path)

def split_info(series):
    # print(series)
    series = series.values[0]
    series = json.loads(series)
    
    if len(series) >= 1:
        series = series[0]
        series = pd.Series(series)
        series = series[['line', 'system']]  
    elif len(series) == 0:
        series = pd.Series({'line':None, 'system':None})
    return series

mrt_line[['line', 'system']] = mrt_line[['lines']].apply(split_info, result_type='expand', axis=1)
mrt_line = mrt_line.drop(['id', 'klass', 'osm_id', 'osm_tags', 'lines', 'osm_metadata'], axis=1)
mrt_line

,length,opening,buildstart,closure,geometry,line,system
0,474,1990,1990,999999.0,"LINESTRING (103.99007 1.35708, 103.99181 1.36099)",Changi Airport Skytrain,Changi Airport Group
1,894,2007,2007,999999.0,"LINESTRING (103.99003 1.35711, 103.99102 1.359...",Changi Airport Skytrain,Changi Airport Group
2,702,2007,2007,999999.0,"LINESTRING (103.98552 1.35388, 103.98510 1.352...",Changi Airport Skytrain,Changi Airport Group
3,454,2007,2007,999999.0,"LINESTRING (103.98382 1.35015, 103.98542 1.35393)",Changi Airport Skytrain,Changi Airport Group
4,434,2007,2007,999999.0,"LINESTRING (103.98739 1.35841, 103.98895 1.36201)",Changi Airport Skytrain,Changi Airport Group
5,2100,2012,2007,999999.0,"LINESTRING (103.86112 1.29221, 103.86124 1.291...",Circle Line,SMRT Trains
6,2023,2007,2003,999999.0,"LINESTRING (103.82244 1.26509, 103.82337 1.265...",Sentosa Express,Sentosa Development Corporation
7,4452,2013,2008,999999.0,"LINESTRING (103.84386 1.28515, 103.84415 1.284...",Downtown Line,SBS Transit
8,6263,1988,1988,999999.0,"LINESTRING (103.84494 1.38178, 103.84420 1.382...",North South Line,SMRT Trains
9,13487,1987,1987,999999.0,"LINESTRING (103.84494 1.38178, 103.84658 1.380...",North South Line,SMRT Trains
